<a href="https://colab.research.google.com/github/delemarchand2020/TestsAzureAI/blob/main/PoC_0_shot_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Classification avec AzureVision OCR + Embedding Azure AI (OpenAI) + Cos-similarité
https://medium.com/@jh.baek.sd/concised-vector-embeddings-for-beginners-guide-3907793e7b10

Autre solution avec LLM:
* https://levelup.gitconnected.com/advanced-document-classification-with-llms-8801eaee3c58

Autre solution avec modèle BERT pré-entrainé et finetuné pour la classification:
* https://medium.com/@j622amilah/zero-shot-classification-with-embeddings-1ebc58f48726
>* https://huggingface.co/mtheo/camembert-base-xnli
>* https://huggingface.co/facebook/bart-large-mnli
* https://joeddav.github.io/blog/2020/05/29/ZSL.html


##Installation des librairies

In [40]:
!pip install -q azure-ai-vision-imageanalysis

In [41]:
!pip install -q gradio

In [42]:
!pip install -q pdf2image

In [43]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [44]:
!pip install -q docx2pdf

In [45]:
!pip install -q openai num2words matplotlib plotly scipy scikit-learn pandas tiktoken

##Chargement des clés Azure et OpenAI

In [46]:
VISION_ENDPOINT = "https://demo-desj-azure-ai.cognitiveservices.azure.com/"

In [47]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [48]:
# Chemin du fichier après le montage de Google Drive
file_path = "/content/gdrive/MyDrive/key.openai.txt"

# Lire le contenu du fichier
with open(file_path, 'r') as file:
    content = file.read()

# Configuration initiale de l'API OpenAI
openai_api_key = content

In [49]:
# Chemin du fichier après le montage de Google Drive
file_path = "/content/gdrive/MyDrive/key.azure.txt"

# Lire le contenu du fichier
with open(file_path, 'r') as file:
    content = file.read()

# Configuration initiale de l'API Azure
VISION_KEY = content

##Chargement d'images de test

In [50]:
!wget https://github.com/delemarchand2020/ExtractDataFromImageDoc/raw/main/recu_masso_photo.png -O recu_masso_photo.png
!wget https://github.com/delemarchand2020/ExtractDataFromImageDoc/raw/main/billet_medecin.png -O billet_medecin.png
!wget https://github.com/delemarchand2020/ExtractDataFromImageDoc/raw/main/fac_ortho.jpg -O fac_ortho.jpg
!wget https://github.com/delemarchand2020/ExtractDataFromImageDoc/raw/main/releve_notes.pdf -O releve_notes.pdf
!wget https://github.com/delemarchand2020/ExtractDataFromImageDoc/raw/main/McGill_Enrolment_Letter_2023-2024.pdf -O McGill_Enrolment_Letter_2023-2024.pdf

--2024-07-10 21:41:02--  https://github.com/delemarchand2020/ExtractDataFromImageDoc/raw/main/recu_masso_photo.png
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/delemarchand2020/ExtractDataFromImageDoc/main/recu_masso_photo.png [following]
--2024-07-10 21:41:02--  https://raw.githubusercontent.com/delemarchand2020/ExtractDataFromImageDoc/main/recu_masso_photo.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3541129 (3.4M) [image/png]
Saving to: ‘recu_masso_photo.png’

recu_masso_photo.pn 100%[===================>]   3.38M  --.-KB/s    in 0.08s   

2024-07-10 21:41:03 (40.2 MB/s) - ‘recu_mas

##Fonctions utiles

###Conversion PDF ou DOCX en image

In [51]:
from pdf2image import convert_from_path
import os

def convert_pdf_to_jpg(file_path):
    # Vérifier si le fichier existe
    if not os.path.isfile(file_path):
        raise FileNotFoundError("Le fichier spécifié n'existe pas.")

    # Vérifier si le fichier est un fichier PDF
    if not file_path.lower().endswith('.pdf'):
        return file_path

    # Chemin du fichier JPEG de sortie
    jpg_path = os.path.splitext(file_path)[0] + ".jpg"

    # Convertir le PDF en une liste d'images PIL
    images = convert_from_path(file_path)

    # Sauvegarder la première image en tant que fichier JPEG
    images[0].save(jpg_path, "JPEG")

    # Retourner le chemin du fichier JPEG
    return jpg_path

In [52]:
from docx2pdf import convert

def convert_docx_to_pdf(file_path):
    # Vérifier si le fichier existe
    if not os.path.isfile(file_path):
        raise FileNotFoundError("Le fichier spécifié n'existe pas.")

    # Vérifier si le fichier est un fichier DOCX
    if not file_path.lower().endswith('.docx'):
        return file_path

    # Chemin du fichier PDF de sortie
    pdf_path = os.path.splitext(file_path)[0] + ".pdf"

    # Convertir le fichier DOCX en PDF
    convert(file_path, pdf_path)

    # Retourner le chemin du fichier PDF
    return convert_pdf_to_jpg(pdf_path)


In [53]:
from PIL import Image
import io

def get_PIL_image(img_path):
  with open(img_path, 'rb') as image_file:
      content = image_file.read()
  return Image.open(io.BytesIO(content))

In [54]:
import numpy as np

def convert_cos_sim_to_angle(cos_sim):
    # Calcul de l'angle en radians
    angle_rad = np.arccos(cos_sim)

    # Conversion de radians en degrés
    angle_deg = np.degrees(angle_rad)

    return round(angle_deg,1)

###Pre-traitement textes

In [55]:
import re

# Définir les expressions régulières pour les dates, les prix, les mots français (y compris les accents) et les nombres
date_regex = re.compile(r'\b\d{1,2}/\d{1,2}/\d{2,4}\b|\b\d{1,2}-\d{1,2}-\d{2,4}\b|\b\d{4}-\d{2}-\d{2}\b|\b\d{2,4}/\d{2}/\d{2,4}\b')
price_regex = re.compile(r'\b\d+[\.,]?\d*\s?(€|EUR|euros?|dollars?|USD|CHF)\b')
french_word_regex = re.compile(r'\b[a-zA-ZàâäéèêëîïôöùûüÿçÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇ]+\b')
number_regex = re.compile(r'\b\d+\b')

# Définir une expression régulière pour les éléments indésirables (ponctuation et autres caractères non désirés)
unwanted_regex = re.compile(r'[^\w\s€|EUR|euros|dollars|USD|CHFàâäéèêëîïôöùûüÿçÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇ\'/-]')

# Fonction pour filtrer le texte
def filter_text(text):
    filtered_elements = []
    words = text.split()

    for word in words:
        # Supprimer les signes de ponctuation et autres caractères non désirés
        clean_word = unwanted_regex.sub('', word)

        if date_regex.match(clean_word) or price_regex.match(clean_word) or french_word_regex.match(clean_word) or number_regex.match(clean_word):
            filtered_elements.append(clean_word)

    return ' '.join(filtered_elements)

In [56]:
# @title Test pré-traitement
# Exemple d'utilisation
text = "Bonjour, je veux acheter une voiture le 12/06/2024 pour 1500 EUR. Mon chat aime bien jouer avec la souris. La date d'aujourd'hui est 2024-06-10. J'ai 2 chats et 1 chien."
filtered_text = filter_text(text)
print(filtered_text)

Bonjour je veux acheter une voiture le 12/06/2024 pour 1500 EUR Mon chat aime bien jouer avec la souris La date d'aujourd'hui est 2024-06-10 J'ai 2 chats et 1 chien


###Appels Azure Vision
https://learn.microsoft.com/fr-ca/python/api/overview/azure/ai-vision-imageanalysis-readme?view=azure-python-preview

In [57]:
import os
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential

# Set the values of your computer vision endpoint and computer vision key
# as environment variables:
try:
    endpoint = VISION_ENDPOINT
    key = VISION_KEY
except KeyError:
    print("Missing environment variable 'VISION_ENDPOINT' or 'VISION_KEY'")
    print("Set them before running this sample.")
    exit()

# Create an Image Analysis client for synchronous operations
client = ImageAnalysisClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(key)
)

In [58]:
# @title Test AzureVision
test=True

if test:
    # Load image to analyze into a 'bytes' object
    with open("billet_medecin.png", "rb") as f:
        image_data = f.read()

    # Extract text (OCR) from an image stream. This will be a synchronously (blocking) call.
    result = client.analyze(
        image_data=image_data,
        visual_features=[VisualFeatures.READ, VisualFeatures.OBJECTS, VisualFeatures.TAGS]
    )

    # Print text (OCR) analysis results to the console
    #print("Image analysis results:")
    #print(" Read:")
    #if result.read is not None:
    #  if result.read.blocks != []:
    #    for line in result.read.blocks[0].lines:
    #        print(f"   Line: '{line.text}', Bounding box {line.bounding_polygon}")
    #        for word in line.words:
    #            print(f"     Word: '{word.text}', Bounding polygon {word.bounding_polygon}, Confidence {word.confidence:.4f}")
    if result.tags is not None:
      print(" Tags:")
      for o in result.tags.list:
          print(f"   tag: '{o}'")

 Tags:
   tag: '{'name': 'text', 'confidence': 0.9999969005584717}'
   tag: '{'name': 'handwriting', 'confidence': 0.9552189111709595}'
   tag: '{'name': 'paper', 'confidence': 0.921732485294342}'
   tag: '{'name': 'ink', 'confidence': 0.8959401249885559}'
   tag: '{'name': 'paper product', 'confidence': 0.8882706165313721}'


In [59]:
# Get text (OCR) analysis results
def get_all_words(result):
    all_words = []
    if result.read.blocks != []:
      for line in result.read.blocks[0].lines:
          for word in line.words:
              all_words.append(word.text)
    return all_words

def get_all_tags(result):
    all_tags = []
    if result.tags is not None:
      all_tags = result.tags.list
    return all_tags

if test:
  print(get_all_words(result))
  print(get_all_tags(result))

['HYGEA', 'clinique', 'médicale', '5100', 'boul.', 'de', 'Maisonneuve', 'O.', 'Bureau', '710', 'Tél.', ':', '514-938-0995', 'Montréal', '(Qc)', 'H4A', '3T2', 'Fax', ':', '514-938-4476', '.Sul,', '31/23', 'Date', ':', 'Marina', 'Grandin', 'Nom', ':', 'Addresse', ':', 'R', 'Custom', 'orthotics', 'Dx', 'Hallux', 'Valgus', 'Geld', 'sman', '87582']
[{'name': 'text', 'confidence': 0.9999969005584717}, {'name': 'handwriting', 'confidence': 0.9552189111709595}, {'name': 'paper', 'confidence': 0.921732485294342}, {'name': 'ink', 'confidence': 0.8959401249885559}, {'name': 'paper product', 'confidence': 0.8882706165313721}]


In [60]:
# Initialize an empty dictionary to store the extracted text
text_dict = {}
tags_dict = {}

def document_texts_detection(img_path, debug=False):
    # Check if the image has already been seen
    if img_path not in text_dict:
        # Load image to analyze into a 'bytes' object
        with open(img_path, "rb") as f:
            image_data = f.read()

        # Extract text (OCR) from an image stream. This will be a synchronously (blocking) call.
        result = client.analyze(
            image_data=image_data,
            visual_features=[VisualFeatures.READ, VisualFeatures.TAGS]
        )
        # Store the extracted text in the dictionary
        text_dict[img_path] = get_all_words(result)
        tags_dict[img_path] = get_all_tags(result)

    # Return the extracted text from the dictionary
    return text_dict[img_path], tags_dict[img_path]


###Appels OpenAI pour embeddings
https://learn.microsoft.com/en-us/azure/ai-services/openai/tutorials/embeddings?tabs=python-new%2Ccommand-line&pivots=programming-language-python

* Voir aussi les compétiteurs : https://blog.voyageai.com/2024/06/10/voyage-multilingual-2-multilingual-embedding-model/
>* A essayer : https://docs.voyageai.com/docs/api-key-and-installation

In [61]:
from openai import OpenAI
import openai

openai.api_key = openai_api_key

client_openai = OpenAI(api_key=openai.api_key)

In [62]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Initialize an empty dictionary to store the embeddings
embeddings_dict = {}

def get_text_embedding(input, model="text-embedding-3-small"):

    if input != None and input != "":
        # Check if the input has already been seen
        if input not in embeddings_dict:
            embeddings_batch_response_openAI = client_openai.embeddings.create(input = [input], model=model)
            # Store the embedding in the dictionary
            embeddings_dict[input] = np.array([embeddings_batch_response_openAI.data[0].embedding])
        # Return the embedding from the dictionary
        return embeddings_dict[input]
    else:
        return np.zeros((1, 1536))

###Tests des appels

In [63]:
texts = document_texts_detection(img_path="recu_masso_photo.png", debug=False)

In [64]:
print(f"Nombre de mots/token détectés: {len(texts)}")

Nombre de mots/token détectés: 2


In [65]:
# @title Texte pour test
texte = """
Université
de Montréal
Denis Lemarchand
4958 avenue grosvenor
montreal QC H3W 2M1
Canada
Programme d'études:
Spécialisation:
Relevé de notes non officiel
Relevé des études de cycles supérieurs
Automne 2022
217510 Informatique (Maîtrise)
Apprentissage automatique (Spécialisation)
Date d'émission:
Matricule:
CPER QC:
2024-01-25
20241211
LEMD02036912
Cours
IFT 6390
Description
Crédits:
Apprent. machine: fondements
Suivis
4.000
Obtenus
4.000
Note Points
A
16.000
Moy.gr.
2-175-1-0 72 Cours à option
Crédits:
Suivis
Moyenne générale trimestrielle
4.000 Total du trimestre
4.000
Obtenus
4.000
Moy.
4.000
Points
16.000
Hiver 2023
Programme d'études:
Spécialisation:
217510 Informatique (Maîtrise)
Apprentissage automatique (Spécialisation)
Cours
Description
Crédits:
IFT 6135
Aprentissage de représent.
Suivis
4.000
Obtenus
4.000
Note Points
A-
14.800
Moy.gr.
2-175-1-0 72 Cours à option
Crédits:
Moyenne générale trimestrielle
3.700 Total du trimestre
Suivis
4.000
Obtenus
4.000
Moy.
4.000
Points
14.800
Automne 2023
Programme d'études:
Spécialisation:
217510 Informatique (Maîtrise)
Apprentissage automatique (Spécialisation)
Cours
Description
Crédits:
IFT 6758
Science des données
Suivis
4.000
Obtenus
4.000
Note Points
A
16.000
Moy.gr.
2-175-1-0 72 Cours à option
Crédits:
Suivis
Moyenne générale trimestrielle
4.000 Total du trimestre
4.000
Obtenus
4.000
Moy.
4.000
Points
16.000
Hiver 2024
Programme d'études:
Spécialisation:
217510 Informatique (Maîtrise)
Intelligence artificielle (Spécialisation)
Cours
Description
Crédits:
IFT 6261
Traitement des connaissances
2-175-1-0 72 Cours à option
Suivis
Obtenus Note
Points
Moy.gr.
Pour être officiel, le relevé de notes doit porter le sceau de l'Université et la signature du Registraire.
Page 1 de 2
"""

###Tests similarité

In [66]:
texte1 = "recevoir une paye pour son métier"
texte2 = "une bulletin de salaire emploi ou travail ou profession ou tâche"
vector1 = get_text_embedding(texte1)
vector2 = get_text_embedding(texte2)
cosine_similarity(vector1, vector2)

array([[0.51359817]])

In [67]:
texte1 = "recevoir une peine pour son crime"
texte2 = "une bulletin de salaire emploi ou travail ou profession ou tâche"
vector1 = get_text_embedding(texte1)
vector2 = get_text_embedding(texte2)
cosine_similarity(vector1, vector2)

array([[0.21518421]])

In [68]:
texte1 = "recevoir une peine pour un crime"
texte2 = "application d'une décision de jugement pénal"
vector1 = get_text_embedding(texte1)
vector2 = get_text_embedding(texte2)
cosine_similarity(vector1, vector2)

array([[0.5661862]])

In [69]:
texte1 = "recevoir une paye pour son métier"
texte2 = "application d'une décision de jugement pénal"
vector1 = get_text_embedding(texte1)
vector2 = get_text_embedding(texte2)
cosine_similarity(vector1, vector2)

array([[0.29377192]])

In [70]:
def my_cosine_similarity(vector1, vector2):
    #Produit scalaire des vecteurs
    scalar_product = np.dot(vector1, vector2.T)
    #Norme euclidienne des vecteurs
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    #Expression analytique du cosinus dans un espace euclidien
    cosine = scalar_product / (norm_vector1 * norm_vector2)
    return cosine

my_cosine_similarity(vector1, vector2)

array([[0.29377192]])

In [71]:
#text-embedding-ada-002, text-embedding-3-small, text-embedding-3-large
model="text-embedding-3-large"
cosine_similarity(get_text_embedding(texte1, model=model), get_text_embedding(texte2, model=model))

array([[0.29377192]])

##Classification

In [72]:
# Define the mapping of sub-types to types
type_mapping = {
    "Relevé de notes officiel universitaire": "Bulletin de notes",
    "Relevé de notes non officiel universitaire": "Bulletin de notes",
    "Bulletin cumulatif de notes": "Bulletin de notes",
    "Cours avec moyenne générale trimestrielle": "Bulletin de notes",
    "Crédits obtenus et notes": "Bulletin de notes",

    "Facture session universitaire": "Preuve de scolarité",
    "Horaire de cours": "Preuve de scolarité",
    "Confirmation d'inscription": "Preuve de scolarité",
    "Statut étudiant temps plein ou temps partiel": "Preuve de scolarité",
    "Attestation d'études": "Preuve de scolarité",

    "Certificat médical": "Billet du médecin",
    "Formulaire d'arrêt de travail": "Billet du médecin",
    "Billet du medecin": "Billet du médecin",
    "Prescription médicale": "Billet du médecin",
    "Ordonnance médicale clinique": "Billet du médecin",

    "Spécimen de chèque": "Spécimen de chèque",
    "Payer à l'ordre de": "Spécimen de chèque",
    "100 dollars": "Spécimen de chèque",
    "Banque": "Spécimen de chèque",
    "MICR code": "Spécimen de chèque"#,

    #"Facture soins dentaires": "Facture",
    #"Facture soin thérapeutique": "Facture",
    #"Facture ou reçu soin massage thérapeutique": "Facture",
    #"Facture et services": "Facture"
}

# Derive types_doc from type_mapping
sous_types_doc = list(type_mapping.keys())
sous_types_doc_original = sous_types_doc.copy()
type_mapping_original = type_mapping.copy()

In [73]:
import json

def classify_text(texte, sous_types_doc, seuil=0.4):
    classification = {}
    insert_autre = False
    total_score = 0

    # Calcul de la similarité pour chaque type de document
    for sous_type in sous_types_doc:
        similarity_score = cosine_similarity(get_text_embedding(texte), get_text_embedding(sous_type))[0][0]
        if similarity_score >= seuil:
            doc_type = type_mapping[sous_type]
            if doc_type in classification:
                classification[doc_type]["cumul des scores de similarité"] += similarity_score
            else:
                classification[doc_type] = {"cumul des scores de similarité": similarity_score}
            total_score += similarity_score
        elif not insert_autre:
            if "Autre" in classification:
                classification["Autre"]["cumul des scores de similarité"] += seuil
            else:
                classification["Autre"] = {"cumul des scores de similarité": seuil}
            total_score += seuil
            insert_autre = True

    # Calcul du pourcentage de chaque score de similarité
    for doc_type, item in classification.items():
        item["part dans la classification (%)"] = round((item["cumul des scores de similarité"] / total_score) * 100, 2)

    # Tri des résultats par score de similarité en ordre décroissant
    sorted_classification = sorted(classification.items(), key=lambda x: x[1]["cumul des scores de similarité"], reverse=True)

    # Conversion de la liste triée en format JSON
    return json.dumps(sorted_classification, indent=4), sorted_classification[0][1]['part dans la classification (%)']

In [74]:
# @title Test classification
img_path = convert_pdf_to_jpg("releve_notes.pdf") #"McGill_Enrolment_Letter_2023-2024.pdf"
img_path = convert_docx_to_pdf(img_path)
texte, _ = document_texts_detection(img_path=img_path)
texte = " ".join(texte)
classification_sorted, best = classify_text(filter_text(texte), sous_types_doc, seuil=0.4963)
json.loads(classification_sorted)

[['Bulletin de notes',
  {'cumul des scores de similarité': 1.6571354241045966,
   'part dans la classification (%)': 62.49}],
 ['Preuve de scolarité',
  {'cumul des scores de similarité': 0.49825607428813773,
   'part dans la classification (%)': 18.79}],
 ['Autre',
  {'cumul des scores de similarité': 0.4963,
   'part dans la classification (%)': 18.72}]]

In [75]:
import numpy as np

def objective_function(seuil, filtered_text, sous_types_doc):
    _, best = classify_text(filtered_text, sous_types_doc, seuil)
    return best

def find_best_seuil(texte, sous_types_doc, seuil_max=0.45, tol=1e-3):
    filtered_text = filter_text(texte)

    low = 0.0
    high = seuil_max

    best_seuil = (low + high) / 2.0
    best_value = objective_function(best_seuil, filtered_text, sous_types_doc)

    while (high - low) > tol:
        mid1 = low + (high - low) / 3
        mid2 = high - (high - low) / 3

        value1 = objective_function(mid1, filtered_text, sous_types_doc)
        value2 = objective_function(mid2, filtered_text, sous_types_doc)

        if value1 > value2:
            high = mid2
        else:
            low = mid1

        best_seuil = (low + high) / 2.0
        best_value = objective_function(best_seuil, filtered_text, sous_types_doc)

    return best_seuil, best_value

best_seuil, best_value = find_best_seuil(texte, sous_types_doc, seuil_max=0.6)
print(f"Le meilleur seuil est {best_seuil} avec une valeur de {best_value}")


Le meilleur seuil est 0.4986384677244059 avec une valeur de 76.87


##Interface UI pour les tests PO-AA

In [76]:
def process_document(img_path, new_type_doc, categorie, parametres, seuil):
    global sous_types_doc_original, type_mapping_original, sous_types_doc, type_mapping

    img_path = convert_pdf_to_jpg(img_path)
    img_path = convert_docx_to_pdf(img_path)
    texts, tags = document_texts_detection(img_path=img_path)
    texts = " ".join(texts)
    image = get_PIL_image(img_path=img_path)

    if "Réinitialiser la liste des sous-types/types" in parametres:
        types_doc_etendu = sous_types_doc_original.copy()
        sous_types_doc = sous_types_doc_original.copy()
        type_mapping = type_mapping_original.copy()
    else:
        types_doc_etendu = sous_types_doc.copy()
        if new_type_doc != "":
            if new_type_doc not in sous_types_doc:
                types_doc_etendu.append(new_type_doc)
                # Add the new type to the type_mapping dictionary
                type_mapping[new_type_doc] = categorie

    if "Nettoyage Texte" in parametres:
        texts = filter_text(texts)

    if "Calcul meilleur seuil" in parametres:
        best_seuil, best = find_best_seuil(texts, types_doc_etendu, seuil_max=float(seuil), tol=1e-3)
        seuil = best_seuil

    analyse, best = classify_text(texts, types_doc_etendu, seuil=seuil)

    return image, seuil, analyse, texts, tags, type_mapping

In [77]:
import gradio as gr

# URL de l'image du diagramme
#diagram_url = "https://diagrams.helpful.dev/d/d:P8TaRIVr-png-base-64-for-mobile"
diagram_url = "https://mermaid.ink/img/pako:eNp9U0FOwzAQ_MrKF0CiPCCHSqj0gIRUiSJOuSz2ElZK7GBvKgriQX1HP8Y6TUtpgFxie2cmuzPxh7HBkSlMoteOvKUbxipiU3rQp8UobLlFL3ATbNeQLjDB7AVjRf3OdeCGyphz22BFPSH4FcXEwQN54Hw8Ri9m9xk7f5OIVjJWxYXehOD8-r2LBI-cJS7G1HnzRM6xr3qB1KIlWJGVKzhftOSvb78Rv7BnIbGnJTdcY2RZ7zpuFIKctA1dQNpXt5u_vXlYt5Qy-46Tdu3OthsfvN1uEjgC0Wp-7_1KY6F7Sl0tRxISebuhcQc76v67k-m0t7oYkolQ00ksfV1xanIxxCEcT9LQokIOVhW7LFid38VAAqXJvobIiWJpfrbRj_9ToDSPR3A49iNlQ0IdKqbBiQNPNU4zKYZEdLZshg1pMorklDNRncHRApahH7hGqP911lyahmKD7PRWfGTh0siL_uylyeM4ekYVzKN_KhQ7Ccu1t6aQ2NGl6VqHsr9EpnjGOh1O547VieHw8wur_0EX?type=png"
cosin_similarity_graph = "https://miro.medium.com/v2/resize:fit:1400/format:webp/1*dyH20eCqb6qTL-gt4nCVzQ.png"

# Interface Gradio
demo = gr.Blocks()

with demo:
    gr.Markdown("# Démo IDP - OCR (ici Azure Vision) - Classification par similarité")
    gr.Image(value=diagram_url, label="")

    threshold_slider = gr.Slider(0.0, 1, value=0.5, label="Seuil similarité")

    outputs=[gr.Image(label="Image du doc", type="pil"), #gr.Textbox(label="Langue détectée"),
            gr.Textbox(label="Meilleur seuil"),
            gr.Json(label="Classification par similarité (0-Shot classification)"),
            gr.Textbox(label="Tous les mots extraits (et filtrés)"),
            gr.Json(label="Tags extraits"),
            gr.Json(label="Regroupement des sous-types par types (actuellement en mémoire)")]

    gr.Interface(
            fn=process_document,
            inputs=[gr.File(label="Fichier", type="filepath", file_types=[".jpg",".pdf",".png",".docx"]),
                    gr.Textbox(label="Ajouter un sous-type de document"),
                    gr.Dropdown(
                        ["Preuve de scolarité", "Bulletin de notes","Billet du médecin", "Spécimen de chèque"], label="Dans la catégorie", value="Preuve de scolarité"),
                    gr.CheckboxGroup(["Nettoyage du texte", "Réinitialiser la liste des sous-types/types", "Calcul meilleur seuil"], label="Paramètres"),
                    threshold_slider],
            outputs=outputs,
            title=None,
            examples= [["fac_ortho.jpg"],["billet_medecin.png"],["recu_masso_photo.png"],["McGill_Enrolment_Letter_2023-2024.pdf"],["releve_notes.pdf"]],
            cache_examples=False, allow_flagging='never', clear_btn=None)

    gr.Image(value=cosin_similarity_graph, label="")

In [78]:
demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f4d9da0653db20ba60.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f4d9da0653db20ba60.gradio.live


##Interface UI pour client

In [ ]:
def process_document(img_path, seuil_max=0.49):
    global sous_types_doc_original, type_mapping_original, sous_types_doc, type_mapping

    img_path = convert_pdf_to_jpg(img_path)
    img_path = convert_docx_to_pdf(img_path)
    texts, tags = document_texts_detection(img_path=img_path)
    texts = " ".join(texts)
    image = get_PIL_image(img_path=img_path)

    texts = filter_text(texts)

    types_doc_etendu = sous_types_doc_original.copy()
    seuil, best = find_best_seuil(texts, types_doc_etendu, seuil_max=seuil_max, tol=1e-3)

    analyse, best = classify_text(texts, types_doc_etendu, seuil=seuil)

    return json.loads(analyse)[0][0], image

In [ ]:
import gradio as gr

# Interface Gradio
poc = gr.Blocks()

with poc:
    gr.Markdown("# PoC IDP - Classification de documents par similarité")

    outputs=[
            gr.Text(label="Classification par similarité (0-Shot classification)"),
            gr.Image(label="Image du doc", type="pil")
            ]

    gr.Interface(
            fn=process_document,
            inputs=[gr.File(label="Fichier", type="filepath", file_types=[".jpg",".pdf",".png",".docx"]),
                    ],
            outputs=outputs,
            title=None,
            examples= [["billet_medecin.png"],["McGill_Enrolment_Letter_2023-2024.pdf"],["releve_notes.pdf"]],
            cache_examples=False, allow_flagging='never', clear_btn=None, submit_btn="Classifier")


In [ ]:
poc.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3f40116f2a722632dc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3f40116f2a722632dc.gradio.live


##interface UI pour traitement en lot

In [ ]:
def predict_class(img_path, seuil_max=0.49):
    global sous_types_doc_original, type_mapping_original, sous_types_doc, type_mapping

    img_path = convert_pdf_to_jpg(img_path)
    img_path = convert_docx_to_pdf(img_path)
    texts, tags = document_texts_detection(img_path=img_path)
    texts = " ".join(texts)
    image = get_PIL_image(img_path=img_path)

    texts = filter_text(texts)

    types_doc_etendu = sous_types_doc_original.copy()
    seuil, best = find_best_seuil(texts, types_doc_etendu, seuil_max=seuil_max, tol=1e-3)

    analyse, best = classify_text(texts, types_doc_etendu, seuil=seuil)

    return json.loads(analyse)[0][0], seuil

In [ ]:
import gradio as gr
import zipfile
import os
import pandas as pd
import matplotlib.pyplot as plt

def unzip_and_get_df(zip_file, batch_path="temp_extracted_files", target_file="0-shot-classif.xlsx"):
    # Create a temporary directory to extract files
    os.makedirs(batch_path, exist_ok=True)

    # Extract the zip file
    with zipfile.ZipFile(zip_file.name, 'r') as zip_ref:
        zip_ref.extractall(batch_path)

    # Locate the specific Excel file
    file_path = os.path.join(batch_path, target_file)

    # Check if the target file exists
    if os.path.exists(file_path):
        # Load the Excel file into a DataFrame
        df = pd.read_excel(file_path)
        return df
    else:
        raise FileNotFoundError(f"File {target_file} not found in the zip archive.")

def highlight(s):
    if s.target != s.predict:
        if (s.target == "Bulletin de notes" and s.predict == "Preuve de scolarité") or (s.target == "Preuve de scolarité" and s.predict == "Bulletin de notes"):
          return ['color:black;background-color:gray'] * len(s)
        else:
          return ['color:white;background-color:purple'] * len(s)
    else:
        return ['color:black;background-color:white'] * len(s)

def process_data(zip_file, seuil_max, batch_path="temp_extracted_files", target_file="0-shot-classif.xlsx"):

    try:
      # Get the dataframe from the unzip_and_get_df function
      df = unzip_and_get_df(zip_file, batch_path, target_file)

      # Combine 'file_name' and 'file_ext' columns to create the list of files
      df['doc_name'] = df['file_name'].astype(str) + '.' + df['file_ext'].astype(str)
    except Exception as e:
      txt_error = f"Error processing zip file: {e}"
      print(txt_error)
      return pd.DataFrame({'Erreur' : [txt_error]}), txt_error

    # Process each file with process_document and update the dataframe
    predicted_classes = []
    best_seuils = []
    for file_name in df['doc_name']:
        file_path = os.path.join(batch_path, file_name)
        try:
          predicted_class, best_seuil = predict_class(file_path, seuil_max=seuil_max)
          predicted_classes.append(predicted_class)
          best_seuils.append(round(best_seuil,3))
        except Exception as e:
          print(f"Error processing file {file_name}: {e}")
          predicted_classes.append("Erreur")

    df['predicted_class'] = predicted_classes
    df['seuil'] = best_seuils
    df['seuil'] = df['seuil'].map('{:,.2f}'.format)
    df['is_correct_class'] = (df['predicted_class'] == df['target_class']).astype(int)

    # Select only the required columns
    df_result = df[['doc_name', 'target_class', 'predicted_class', 'seuil']]
    df_result.columns = ['doc_name', 'target', 'predict', 'seuil']

    # Calculate classification accuracy
    overall_accuracy = df['is_correct_class'].mean() * 100
    class_accuracy = df.groupby('target_class')['is_correct_class'].mean() * 100

    textstr = f'Taux de bonne classification au global : {overall_accuracy:.2f}%\n'
    for cls, acc in class_accuracy.items():
        textstr += f'         - Taux pour {cls}: {acc:.2f}%\n'

    return df_result.style.apply(highlight, axis=1), textstr

poc = gr.Blocks()

with poc:
    gr.Markdown("# PoC IDP - Classification de documents par similarité - traitement en lot")

    input_zip = gr.File(label="Fichier ZIP contenant les échantillons à classer")
    threshold_slider = gr.Slider(0.0, 1, value=0.31, label="Seuil similarité max")
    submit_btn = gr.Button("Classifier")

    output_df = gr.Dataframe(label="")
    output_text = gr.Text(label="")

    submit_btn.click(
        fn=process_data,
        inputs=[input_zip, threshold_slider],
        outputs=[output_df, output_text]
    )


In [ ]:
poc.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://508725e00ebb5bb049.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://508725e00ebb5bb049.gradio.live


In [ ]:
!rm -rf temp_extracted_files

In [ ]:
!zip -r 0-shot-classif.zip temp_extracted_files